<a href="https://colab.research.google.com/github/dscho15/DeepLearning/blob/main/Lecture1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture 1

This lecture, we implement a KNN classifier to recognize images from a known dataset: MNIST.



In [24]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [10]:
import torch
from torchvision import datasets

mnist_train_dataset = datasets.MNIST('', train=True, download=True);
mnist_test_dataset = datasets.MNIST('', train=False, download=True); 

# The tensor sizes of the data labels etc.

mnist_train_labels = mnist_train_dataset.targets.numpy();
mnist_train_data = mnist_train_dataset.data.numpy().reshape((-1, 28*28));

mnist_test_labels = mnist_test_dataset.targets.numpy();
mnist_test_data = mnist_test_dataset.data.numpy().reshape((-1, 28*28))

print("The train labls and data: ", mnist_train_labels.shape, mnist_train_data.shape)
print("The test labels and data: ", mnist_test_labels.shape, mnist_test_data.shape)

The train labls and data:  (60000,) (60000, 784)
The test labels and data:  (10000,) (10000, 784)


In [11]:
cifar10_train_dataset = datasets.CIFAR10('../', train=True, download=True);
cifar10_test_dataset = datasets.CIFAR10('../', train=False, download=True);

cifar10_train_labels = np.array(cifar10_train_dataset.targets)
cifar10_train_data = np.array(cifar10_train_dataset.data)

cifar10_test_labels = np.array(cifar10_test_dataset.targets)
cifar10_test_data = np.array(cifar10_test_dataset.data)

print("The train labls and data: ", cifar10_train_labels.shape, cifar10_train_data.shape)
print("The test labels and data: ", cifar10_test_labels.shape, cifar10_test_data.shape)

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140]);

cifar10_train_data_gray = rgb2gray(cifar10_train_data).reshape((-1, 32*32))
cifar10_test_data_gray = rgb2gray(cifar10_test_data).reshape((-1, 32*32))

print("The train labls and data(gray): ", cifar10_train_labels.shape, cifar10_train_data_gray.shape)
print("The test labels and data(gray): ", cifar10_test_labels.shape, cifar10_test_data_gray.shape)


Files already downloaded and verified
Files already downloaded and verified
The train labls and data:  (50000,) (50000, 32, 32, 3)
The test labels and data:  (10000,) (10000, 32, 32, 3)
The train labls and data(gray):  (50000,) (50000, 1024)
The test labels and data(gray):  (10000,) (10000, 1024)


In [13]:
def KNN(train_data, train_label, test_data, test_label, shuffle = [True, 10000], k = [1, 5, 10, 25], p=[1, 2]):

  methods = []
  results = []
  confusion_matrices = []

  if shuffle[0] == True:
    indices = np.random.permutation(train_data.shape[0])
    train_data = train_data[indices]
    train_label = train_label[indices]
    train_data = train_data[:shuffle[1]]
    train_label = train_label[:shuffle[1]]

  for i in p:

    for j in k:

      # Make a classifier
      KNC = KNeighborsClassifier(j, algorithm='brute', n_jobs = -1, p=i)

      # Fit the classifiers
      KNC.fit(train_data, train_label) 

      # Get the results
      prediction = KNC.predict(test_data)

      # How well do we perform with k = 5?
      results.append(np.sum(prediction == test_label)/test_label.shape[0])

      # Get the confusion matrix
      confusion_matrices.append(confusion_matrix(test_label, prediction))

      methods.append([i, j])

  return results, confusion_matrices, methods

In [14]:
mnist_results, mnist_cfmatrix, mnist_method = KNN(mnist_train_data, mnist_train_labels, mnist_test_data, mnist_test_labels)
cifar10_results, cifar10_cfmatrix, cifar10_method = KNN(cifar10_train_data_gray, cifar10_train_labels, cifar10_test_data_gray, cifar10_test_labels)

In [ ]:
print("MNIST results, only KNN: \n", mnist_results)
print("CIFAR10 results, only KNN: \n", cifar10_results)

In [22]:
def PCA_KNN(train_data, train_label, test_data, test_label, comp=[5, 10, 25, 30]):

  results, cf_matrix, met = [], [], []

  for i in tqdm(range(len(comp))):
    
    # Create instance
    pca = PCA(comp[i])

    # Create transform and get it
    pca_train = pca.fit_transform(train_data)
    pca_test = pca.transform(test_data)

    # Parse to KNN
    results_knn, confusion_matrices_knn, methods_knn = KNN(pca_train, train_label, pca_test, test_label)

    results.append(results_knn)
    cf_matrix.append(confusion_matrices_knn)
    met.append(methods_knn)

  return results, cf_matrix, met, comp

In [23]:
mnist_results, mnist_cf_matrix, mnist_met, mnist_comp = PCA_KNN(mnist_train_data, mnist_train_labels, mnist_test_data, mnist_test_labels)
cifar_results, cifar_cf_matrix, cifar_met, cifar_comp = PCA_KNN(cifar10_train_data_gray, cifar10_train_labels, cifar10_test_data_gray, cifar10_test_labels)

100%|██████████| 4/4 [02:54<00:00, 43.75s/it]


In [27]:
print("MNIST results, only KNN: \n", np.max(mnist_results))
print("CIFAR10 results, only KNN: \n", np.max(cifar10_results))

MNIST results, only KNN: 
 0.9517
CIFAR10 results, only KNN: 
 0.2889


In [30]:
def TSNE_EMBEDDING(train_data, train_label):

  # Get the embedding, then make a scatter plot
  train_embedding = TSNE(n_components=2).fit_transform(train_data)

  colors = train_label / 10

  fig = plt.figure(figsize=(8,8))
  plt.scatter(x, y, c=colors)  
  plt.show()

In [ ]:
TSNE_EMBEDDING(mnist_train_data, mnist_train_labels)